In [1]:
import os
import glob

In [2]:
def translate_result(output_path):
    result = None


    print(result)
    return result

In [3]:
subfolders = [ f.path for f in os.scandir("data") if f.is_dir() ]

In [4]:
print(subfolders)

['data/simple_modified_2024_03_01_sim_dir', 'data/steeringwheel', 'data/simple_10000_load_sim_dir', 'data/simpleexample', 'data/simple_sim_dir']


In [5]:
for proj in subfolders:
    #print(proj)
    sim_dir = os.path.join(proj, "sim_dir")
    cpu_clusters = [ f.path for f in os.scandir(sim_dir) if f.is_dir() ]
    print(cpu_clusters)
    

['data/simple_modified_2024_03_01_sim_dir/sim_dir/ECU1-SoC1-CortexA53']
['data/steeringwheel/sim_dir/ECU1-SoC1-CortexA53', 'data/steeringwheel/sim_dir/ECU1-SoC1-CortexA72']
['data/simple_10000_load_sim_dir/sim_dir/ECU1-SoC1-CortexA53']
['data/simpleexample/sim_dir/ECU1-SoC1-CortexA53', 'data/simpleexample/sim_dir/ECU1-SoC1-CortexA72']
['data/simple_sim_dir/sim_dir/ECU1-SoC1-CortexA53']


In [6]:
#cpu_clusters = [ f.path for f in os.scandir("data/simple_sim_dir/sim_dir") if f.is_dir() ]
#cpu_clusters = [ f.path for f in os.scandir("data/simple_modified_2024_03_01_sim_dir/sim_dir") if f.is_dir() ]
cpu_clusters = [ f.path for f in os.scandir("data/simple_10000_load_sim_dir/sim_dir") if f.is_dir() ]
print(cpu_clusters)

['data/simple_10000_load_sim_dir/sim_dir/ECU1-SoC1-CortexA53']


In [7]:
from bigtree import Node
from bigtree.utils.exceptions import TreeError
from collections import deque

root = Node("0", event_time="")
event_to_node = {}
event_to_node[""] = root
fn_to_node = {}
fn_to_node["0"] = root

FUNCTION_ENTRY_VALUE = "ust.function.entry"
FUNCTION_EXIT_VALUE = "ust.function.exit"

def add_function_graph(event_time, function_type, this_fn, call_site):
    print("calling add_function_graph event_time: {}, function_type: {}, this_fn: {}, call_site: {}".format(event_time, function_type, this_fn, call_site))
    if function_type == FUNCTION_ENTRY_VALUE:
        try:
            new_node = Node(this_fn, event_time=event_time, parent=fn_to_node[call_site])
            event_to_node[event_time] = new_node
            fn_to_node[this_fn] = new_node
            print(new_node)
        except TreeError as te:
            print(te)
    elif function_type == FUNCTION_EXIT_VALUE:
        print("{} exited".format(this_fn))

running_functions = deque()
init_time = 0
busy_time = 0
simulation_time = 700000000
idle_time = 700000000
busy_start_time = 0
last_exit_time = 0

def process_function_event(event_time, function_type, this_fn, call_site):
    global init_time
    global running_functions
    global busy_time
    global busy_start_time
    global last_exit_time
    #print("calling process_function_event event_time: {}, function_type: {}, this_fn: {}, call_site: {}".format(event_time, function_type, this_fn, call_site))
    if function_type == FUNCTION_ENTRY_VALUE:
        if init_time == 0:
            init_time = int(event_time)
        if len(running_functions) == 0:
            busy_start_time = int(event_time)
        running_functions.append(int(event_time))
    elif function_type == FUNCTION_EXIT_VALUE:
        running_functions.pop()
        if len(running_functions) == 0:
            busy_time += int(event_time) - busy_start_time
        last_exit_time = int(event_time)


def parse_function_events_per_core(core_path):
    # parse function_activity_gantt0.csv
    #files = [f for f in os.scandir(core_path) if os.path.isfile(f)]
    #print(files)
    gantt_file = glob.glob(core_path + "/function_activity_gantt0.csv")[0]
    print(gantt_file)
    with open(gantt_file, "r") as file:
        while line := file.readline():
            line = line.rstrip()
            fields = line.split(';')
            is_event_line = fields[0].isnumeric()
            if not is_event_line:
                continue
            if len(fields) < 8:
                continue
            event_time = fields[0]
            function_type = fields[1]
            this_fn = fields[4].split('=')[-1]
            call_site = fields[5].split('=')[-1]
            add_function_graph(event_time, function_type, this_fn, call_site)
    #root.hshow()

def parse_function_activity_per_core(core_path):
    activity_file = glob.glob(core_path + "/function_activity0.csv")[0]
    print(activity_file)
    time_stamps_algo1 = []
    time_stamps_algo5 = []
    with open(activity_file, "r") as file:
        while line := file.readline():
            line = line.rstrip()
            fields = line.split(';')
            is_event_line = fields[0].isnumeric()
            if not is_event_line:
                continue
            if len(fields) < 7:
                continue
            event_time = fields[0]
            function_type = fields[1]
            if "Rte_IWrite_SWC1_R1_Algo1_swc1PP1_R1Data" in function_type:
                #print("R1_Algo1: {}".format(event_time))
                time_stamps_algo1.append(int(event_time))
            if "Rte_IWrite_SWC2_R5_Algo5_SWC2PP1_R5Data" in function_type:
                #print("R5_Algo5: {}".format(event_time))
                time_stamps_algo5.append(int(event_time))
    prev_time = 0
    print("algo1")
    for idx, time in enumerate(time_stamps_algo1):
        print("{} and prev gap: {}. Stamp2 {}, Stamnp1 {}".format(idx, time - prev_time, time, prev_time))
        prev_time = time
    prev_time = 0
    print("algo5")
    for idx, time in enumerate(time_stamps_algo5):
        print("{} and prev gap: {}".format(idx, time - prev_time))
        prev_time = time

def parse_idle_per_core(core_path):
    global busy_time
    global idle_time
    global last_exit_time
    global init_time
    gantt_file = glob.glob(core_path + "/function_activity_gantt0.csv")[0]
    print(gantt_file)
    
    with open(gantt_file, "r") as file:
        while line := file.readline():
            line = line.rstrip()
            fields = line.split(';')
            is_event_line = fields[0].isnumeric()
            if not is_event_line:
                continue
            if len(fields) < 8:
                continue
            event_time = fields[0]
            function_type = fields[1]
            this_fn = fields[4].split('=')[-1]
            call_site = fields[5].split('=')[-1]
            process_function_event(event_time, function_type, this_fn, call_site)
    idle_time -= busy_time
    system_time = init_time + simulation_time - last_exit_time
    idle_time -= system_time
    print("busy_time: {}, idle_time: {}, system_time: {}".format(busy_time, idle_time, system_time))
    print("idle percentage: {}, task busy percentage: {}, system percentage: {}".format(idle_time/simulation_time * 100, busy_time/simulation_time * 100, system_time/simulation_time * 100))

for cpu_cluster in cpu_clusters:
    cores_dir = os.path.join(cpu_cluster, "system_analyzer")
    print(cores_dir)
    core_folders = [ f.path for f in os.scandir(cores_dir) if f.is_dir() ]
    print(core_folders)
    # for each cpu core, get the function activities
    #for core_path in core_folders:
    #    parse_function_events_per_core(core_path)
    #parse_function_events_per_core(core_folders[0])
    parse_function_activity_per_core(core_folders[1])
    parse_idle_per_core(core_folders[1])

data/simple_10000_load_sim_dir/sim_dir/ECU1-SoC1-CortexA53/system_analyzer
['data/simple_10000_load_sim_dir/sim_dir/ECU1-SoC1-CortexA53/system_analyzer/ECU1_SoC1_bare-metal-top.Cluster0.ARMV8_Cluster.CPU.PV.cpu1.core', 'data/simple_10000_load_sim_dir/sim_dir/ECU1-SoC1-CortexA53/system_analyzer/ECU1_SoC1_bare-metal-top.Cluster0.ARMV8_Cluster.CPU.PV.cpu0.core']
data/simple_10000_load_sim_dir/sim_dir/ECU1-SoC1-CortexA53/system_analyzer/ECU1_SoC1_bare-metal-top.Cluster0.ARMV8_Cluster.CPU.PV.cpu0.core/function_activity0.csv
algo1
0 and prev gap: 112616325. Stamp2 112616325, Stamnp1 0
1 and prev gap: 454249. Stamp2 113070574, Stamnp1 112616325
2 and prev gap: 76722907. Stamp2 189793481, Stamnp1 113070574
3 and prev gap: 156144. Stamp2 189949625, Stamnp1 189793481
4 and prev gap: 73852762. Stamp2 263802387, Stamnp1 189949625
5 and prev gap: 158329. Stamp2 263960716, Stamnp1 263802387
6 and prev gap: 73832765. Stamp2 337793481, Stamnp1 263960716
7 and prev gap: 156144. Stamp2 337949625, Stamnp